In [ ]:
import os
from dotenv import load_dotenv

from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import pandas as pd
from tqdm import tqdm

import tiktoken
from pydantic import BaseModel, Field

from typing import List, Optional

from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores.neo4j_vector import (
    Neo4jVector,
)

from neo4j import Result
from typing import Dict, Any
from langchain.chains import RetrievalQA
from bert_score import score as bert_score
import string

In [ ]:
load_dotenv() 

In [ ]:
graph = Neo4jGraph(refresh_schema=True, database="neo4j")    

In [ ]:
# Hàm đếm tokens
def num_tokens_from_string(string: str, model_name: str = "gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    return len(encoding.encode(string))

In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

llm_transformer = LLMGraphTransformer(
  llm=llm, 
  node_properties=["description"],
  relationship_properties=["description"]
)

def process_text(text: str) -> List[Document]:
    doc = Document(page_content=text)
    return llm_transformer.convert_to_graph_documents([doc])

# Process data

**Process Text data**

In [ ]:
# test
sample_text = ""
with open("data/sample_text.txt", "r", encoding="utf-8") as f:
    sample_text = f.read()

sample_doc = process_text(text=sample_text)

print(sample_doc)

In [ ]:
graph.add_graph_documents(
    sample_doc,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
documents = []

for i in range(1, 64):
    chunkDoc = ""
    chunkFileName = "p" + str(i) + ".txt"
    with open("data/quydinhdaotaothacsi/" + chunkFileName, "r", encoding="utf-8") as f:
        chunkDoc = f.read()
    documents.append(chunkDoc)    
    chunkDocProcessed = process_text(text=chunkDoc)
    graph.add_graph_documents(
    chunkDocProcessed,
    baseEntityLabel=True,
    include_source=True
)
    
print(len(documents))

In [ ]:
MAX_WORKERS = 2
NUM_CHUNK = 63

def process_chunk_file(i):
    chunkFileName = f"p{i}.txt"
    file_path = f"data/quydinhdaotaothacsi/{chunkFileName}"
    with open(file_path, "r", encoding="utf-8") as f:
        chunkDoc = f.read()
    return process_text(text=chunkDoc)

graph_documents = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [
        executor.submit(process_chunk_file, i)
        for i in range(1, NUM_CHUNK + 1)
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        chunkDocProcessed = future.result()
        graph_documents.extend(chunkDocProcessed)

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

## De-duplication

In [ ]:
# Create and store vector embeddings
vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model="text-embedding-3-small"),
    node_label='__Entity__',
    text_node_properties=['id', 'description'],
    embedding_node_property='embedding'
)

In [ ]:
gds = GraphDataScience(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)

In [ ]:
G, result = gds.graph.project(
    "entities",                   # Graph name
    "__Entity__",                 # Node projection
    "*",                          # Relationship projection
    nodeProperties=["embedding"]  # Configuration parameters
)

In [ ]:
similarity_threshold = 0.95

gds.knn.mutate(
  G,
  nodeProperties=['embedding'],
  mutateRelationshipType= 'SIMILAR',
  mutateProperty= 'score',
  similarityCutoff=similarity_threshold
)

In [ ]:
gds.wcc.write(
    G,
    writeProperty="wcc",
    relationshipTypes=["SIMILAR"]
)

In [ ]:
word_edit_distance = 3
potential_duplicate_candidates = graph.query(
    """MATCH (e:`__Entity__`)
    WHERE size(e.id) > 3 // longer than 3 characters
    WITH e.wcc AS community, collect(e) AS nodes, count(*) AS count
    WHERE count > 1
    UNWIND nodes AS node
    // Add text distance
    WITH distinct
      [n IN nodes WHERE apoc.text.distance(toLower(node.id), toLower(n.id)) < $distance 
                  OR node.id CONTAINS n.id | n.id] AS intermediate_results
    WHERE size(intermediate_results) > 1
    WITH collect(intermediate_results) AS results
    // combine groups together if they share elements
    UNWIND range(0, size(results)-1, 1) as index
    WITH results, index, results[index] as result
    WITH apoc.coll.sort(reduce(acc = result, index2 IN range(0, size(results)-1, 1) |
            CASE WHEN index <> index2 AND
                size(apoc.coll.intersection(acc, results[index2])) > 0
                THEN apoc.coll.union(acc, results[index2])
                ELSE acc
            END
    )) as combinedResult
    WITH distinct(combinedResult) as combinedResult
    // extra filtering
    WITH collect(combinedResult) as allCombinedResults
    UNWIND range(0, size(allCombinedResults)-1, 1) as combinedResultIndex
    WITH allCombinedResults[combinedResultIndex] as combinedResult, combinedResultIndex, allCombinedResults
    WHERE NOT any(x IN range(0,size(allCombinedResults)-1,1)
        WHERE x <> combinedResultIndex
        AND apoc.coll.containsAll(allCombinedResults[x], combinedResult)
    )
    RETURN combinedResult
    """, params={'distance': word_edit_distance})

In [ ]:
system_prompt = """Xác định các thực thể trùng lặp trong danh sách và quyết định thực thể nào trong số chúng nên được hợp nhất.
Các thực thể có thể hơi khác nhau về định dạng hoặc nội dung, nhưng về cơ bản đều đề cập đến cùng một thứ. Sử dụng các kỹ năng phân tích của bạn để xác định các bản sao.

Sau đây là các quy tắc để xác định các bản sao:
1. Các thực thể có sự khác biệt nhỏ về kiểu chữ nên được coi là bản sao.
2. Các thực thể có định dạng khác nhau nhưng cùng nội dung nên được coi là bản sao.
3. Các thực thể đề cập đến cùng một đối tượng hoặc khái niệm trong thế giới thực, ngay cả khi được mô tả khác nhau, nên được coi là bản sao.
4. Nếu nó đề cập đến các số, ngày hoặc sản phẩm khác nhau, thì không được hợp nhất kết quả
"""
user_template = """
Dưới đây là danh sách các thực thể cần xử lý:
{entities}

Xác định các mục trùng lặp, hợp nhất chúng và cung cấp danh sách đã hợp nhất.
"""

In [ ]:
class DuplicateEntities(BaseModel):
    entities: List[str] = Field(
        description="Các thực thể đại diện cho cùng một đối tượng hoặc thực thể trong thế giới thực và cần được hợp nhất"
    )


class Disambiguate(BaseModel):
    merge_entities: Optional[List[DuplicateEntities]] = Field(
        description="Danh sách các thực thể đại diện cho cùng một đối tượng hoặc thực thể trong thế giới thực và cần được hợp nhất"
    )


extraction_llm = ChatOpenAI(model_name="gpt-4o").with_structured_output(
    Disambiguate
)

extraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            user_template,
        ),
    ]
)

In [ ]:
extraction_chain = extraction_prompt | extraction_llm

def entity_resolution(entities: List[str]) -> Optional[List[List[str]]]:
    return [
        el.entities
        for el in extraction_chain.invoke({"entities": entities}).merge_entities
    ]

In [ ]:
merged_entities = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submitting all tasks and creating a list of future objects
    futures = [
        executor.submit(entity_resolution, el['combinedResult'])
        for el in potential_duplicate_candidates
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        to_merge = future.result()
        if to_merge is not None:
            merged_entities.extend(to_merge)

In [ ]:
print(merged_entities)

In [ ]:
graph.query("""
UNWIND $data AS candidates
CALL {
  WITH candidates
  MATCH (e:__Entity__) WHERE e.id IN candidates
  RETURN collect(e) AS nodes
}
CALL apoc.refactor.mergeNodes(nodes, {properties: {
    description:'combine',
    `.*`: 'discard'
}})
YIELD node
RETURN count(*)
""", params={"data": merged_entities})

# Constructing and Summarizing Communities

In [ ]:
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [ ]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

In [ ]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

In [ ]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

In [ ]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:MENTIONS]-(d:Document)
WITH c, count(distinct d) AS rank
SET c.community_rank = rank;
""")

In [ ]:
community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

In [ ]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")

In [ ]:
community_template = """Dựa trên các nút và mối quan hệ được cung cấp thuộc về cùng một cộng đồng đồ thị, tạo bản tóm tắt bằng ngôn ngữ tự nhiên về thông tin được cung cấp:
{community_info}

Tóm tăt:"""

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Với một bộ ba đầu vào, tạo ra bản tóm tắt thông tin. Không có phần mở đầu.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [ ]:
def prepare_string(data):
    nodes_str = "Nodes are:n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}n"

    rels_str = "Relationships are:n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}n"

    return nodes_str + "n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

In [ ]:
summaries = []
MAX_WORKERS = 2
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

In [ ]:
print(summaries)

In [ ]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

### Local Retrieval

In [ ]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    print("Fulltext index created failed.")
    pass

# Close the driver connection
driver.close()

** Extract entities from question **

In [ ]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="Tất cả các thực thể trong văn bản",
    )

extract_entities_from_question_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Trích xuất thực thể từ văn bản.",
        ),
        (
            "human",
            "Sử dụng định dạng đã cho để trích xuất thông tin các thực thể từ những nội dung sau, trả kết quả dưới dạng danh sách, nếu là một thực thể nhiều thông tin, hoặc là một cụm danh từ tiếng việt , hãy chia ra thành nhiều thực thể con"
            "input: {question}",
        ),
    ]
)

entity_chain = extract_entities_from_question_prompt | llm.with_structured_output(Entities)

def extract_entities_from_question(question):
    entity_chain.invoke(input=question)

### Local query


In [ ]:
BASE_LOCAL_QUERY = """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """
LOCAL_QUERY_WITH_DESSCRIPTION = """
            CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN
                node.id + ' (desc: ' + coalesce(node.description, '') + ')' + ' - ' +
                type(r) + ' -> ' +
                neighbor.id + ' (desc: ' + coalesce(neighbor.description, '') + ')' AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN
                neighbor.id + ' (desc: ' + coalesce(neighbor.description, '') + ')' + ' - ' +
                type(r) + ' -> ' +
                node.id + ' (desc: ' + coalesce(node.description, '') + ')' AS output
            }
            RETURN output LIMIT 100
            """

In [ ]:
# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            LOCAL_QUERY_WITH_DESSCRIPTION,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result


In [ ]:
# Sử dụng OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_index = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="embedding",
    search_type="hybrid",
)

vector_retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 3})

vector_graph_chain = RetrievalQA.from_chain_type(
        llm, 
        retriever = vector_retriever, 
        verbose=False,
        return_source_documents=False,
    )

In [ ]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
    {graph_data}
    vector data:
    {"#Document ". join(vector_data)}
    """
    return final_data

In [ ]:
graph_retriever("Các hình thức thi kết thúc môn học là gì?")

In [ ]:
template = """Trả lời câu hỏi dựa trên ngữ cảnh được cung cấp, sẽ có thông tin embedding trong ngữ cảnh:
{context}
Thông tin trong desc là mô tả của thực thể trong ngữ cảnh. Có một số quan hệ được mô tả bằng tiếng Anh, hãy thêm chúng vào ngữ cảnh.
Câu hỏi: {question}
Sử dụng ngôn ngữ tự nhiên và trả lời ngắn gọn. Không thêm vào các thông tin như có liên quan đến cộng đồng...
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

def graph_answer(input):
    return chain.invoke(input=input)

In [ ]:
graph_answer("Các môn học thuộc khối kiến thức bổ sung là gì?")

### Global Retrieval


In [ ]:
def db_query(cypher: str, params: Dict[str, Any] = {}) -> pd.DataFrame:
    """Executes a Cypher statement and returns a DataFrame"""
    return driver.execute_query(
        cypher, parameters_=params, result_transformer_=Result.to_df
    )

In [ ]:
MAP_SYSTEM_PROMPT = """
---Vai trò---

Bạn là một trợ lý hữu ích, phản hồi các câu hỏi liên quan đến dữ liệu trong các bảng được cung cấp.


---Mục tiêu---

Tạo ra một phản hồi gồm danh sách các điểm chính nhằm trả lời câu hỏi của người dùng, tóm tắt tất cả thông tin liên quan trong các bảng dữ liệu đầu vào.

Bạn nên sử dụng dữ liệu được cung cấp trong các bảng dữ liệu dưới đây làm bối cảnh chính để tạo phản hồi.
Nếu bạn không biết câu trả lời hoặc nếu các bảng dữ liệu đầu vào không chứa đủ thông tin để đưa ra câu trả lời, chỉ cần nói như vậy. Không được bịa ra bất kỳ thông tin nào.

Mỗi điểm chính trong phản hồi phải bao gồm các yếu tố sau:
- Mô tả: Một mô tả đầy đủ về điểm đó.
- Điểm quan trọng: Một số nguyên trong khoảng từ 0-100 thể hiện mức độ quan trọng của điểm đó trong việc trả lời câu hỏi của người dùng. Phản hồi kiểu “Tôi không biết” sẽ có điểm là 0.

Phản hồi phải được định dạng JSON như sau:
{{
    "points": [
        {{"description": "Mô tả điểm 1 [Dữ liệu: Báo cáo (các mã báo cáo)]", "score": giá_trị_điểm}},
        {{"description": "Mô tả điểm 2 [Dữ liệu: Báo cáo (các mã báo cáo)]", "score": giá_trị_điểm}}
    ]
}}

Phản hồi phải giữ nguyên ý nghĩa gốc và cách sử dụng các động từ tình thái như “sẽ”, “có thể” .

Các điểm có dữ liệu hỗ trợ nên liệt kê các báo cáo liên quan làm tài liệu tham khảo như sau:
"Đây là một câu ví dụ được hỗ trợ bởi các tài liệu tham khảo [Dữ liệu: Báo cáo (các mã báo cáo)]"

**Không liệt kê nhiều hơn 5 mã báo cáo trong một tài liệu tham khảo**. Thay vào đó, hãy liệt kê 5 mã báo cáo liên quan nhất và thêm “+more” để cho biết còn nhiều hơn nữa.

Ví dụ:
"Người X là chủ sở hữu của Công ty Y và bị cáo buộc nhiều sai phạm [Dữ liệu: Báo cáo (2, 7, 64, 46, 34, ...)]. Anh ta cũng là CEO của công ty X [Dữ liệu: Báo cáo (1, 3)]"

trong đó 1, 2, 3, 7, 34, 46, và 64 là mã (không phải chỉ số) của các báo cáo dữ liệu liên quan trong bảng được cung cấp.

Không bao gồm thông tin nếu không có bằng chứng hỗ trợ từ dữ liệu.

---Các bảng dữ liệu---

{context_data}
"""


map_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            MAP_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

map_chain = map_prompt | llm | StrOutputParser()

In [ ]:
REDUCE_SYSTEM_PROMPT = """
---Vai trò---

Bạn là một trợ lý hữu ích, trả lời các câu hỏi về một tập dữ liệu bằng cách tổng hợp các quan điểm từ nhiều nhà phân tích khác nhau.


---Mục tiêu---

Tạo ra một phản hồi với độ dài và định dạng mục tiêu nhằm trả lời câu hỏi của người dùng, đồng thời tóm tắt tất cả các báo cáo từ nhiều nhà phân tích, mỗi người tập trung vào các phần khác nhau của tập dữ liệu.

Lưu ý rằng các báo cáo của nhà phân tích được cung cấp dưới đây **được sắp xếp theo thứ tự quan trọng giảm dần**.

Nếu bạn không biết câu trả lời hoặc nếu các báo cáo được cung cấp không chứa đủ thông tin để đưa ra câu trả lời, chỉ cần nói như vậy. Không được bịa ra bất kỳ thông tin nào.

Phản hồi cuối cùng nên loại bỏ tất cả thông tin không liên quan từ các báo cáo của nhà phân tích và tổng hợp thông tin đã được làm sạch thành một câu trả lời toàn diện, giải thích tất cả các điểm chính và các hàm ý, phù hợp với độ dài và định dạng của phản hồi.

Hãy thêm các phần và bình luận vào phản hồi nếu phù hợp với độ dài và định dạng. Định dạng phản hồi bằng **markdown**.

Phản hồi phải giữ nguyên ý nghĩa gốc và cách sử dụng các động từ tình thái như “sẽ”, “có thể”.

Phản hồi cũng phải giữ lại tất cả các tham chiếu dữ liệu đã được nêu trong các báo cáo của nhà phân tích, nhưng **không được đề cập đến vai trò của các nhà phân tích trong quá trình phân tích**.

**Không liệt kê nhiều hơn 5 mã báo cáo trong một tham chiếu**. 


---Độ dài và định dạng phản hồi mục tiêu---

{response_type}


---Báo cáo của các nhà phân tích---

{report_data}

Hãy thêm các phần và bình luận vào phản hồi nếu phù hợp với độ dài và định dạng. Định dạng phản hồi bằng markdown.
"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            REDUCE_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)
reduce_chain = reduce_prompt | llm | StrOutputParser()

In [ ]:
community_embedding = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    node_label="__Community__",
    text_node_properties=["summary"],
    embedding_node_property="embedding",
    index_name="fulltext_community_summary",
    search_type="hybrid",
)

vector_community_retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
def global_retriever(query: str, level: int = 0, response_type: str = "nhiều đoạn văn bản") -> str:
    
    # community_data = graph.query(
    #     """
    #     MATCH (c:__Community__)
    #     WHERE c.level = $level and c.summary is not null
    #     RETURN c.summary AS output limit 20
    #     """,
    #     params={"level": level},
    # )
    query_embedding = embeddings.embed_query(query)
    community_data = graph.query(
        """
        WITH $query_embedding AS queryEmbedding
        MATCH (c:__Community__)
        WHERE c.embedding IS NOT NULL and (c.level = 0 or c.level = 1)
        WITH c, vector.similarity.cosine(queryEmbedding, c.embedding) AS score
        RETURN c.summary AS output
        ORDER BY score DESC
        LIMIT 10
        """,
        params={"query_embedding": query_embedding},
    )
    
    def process_community(community):
        return map_chain.invoke({"question": query, "context_data": community["output"]})

    # Dùng ThreadPoolExecutor để chạy song song
    intermediate_results = []
    with ThreadPoolExecutor(max_workers=5) as executor:  # có thể chỉnh max_workers tuỳ môi trường
        futures = [executor.submit(process_community, comm) for comm in community_data]
        for f in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
            intermediate_results.append(f.result())

    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

def global_answer(question: str):
    return global_retriever(query=question)

In [ ]:
print(global_answer("Tóm tắt thông tin về chương trình đào tạo thạc sĩ?"))

**Determine query type**

In [ ]:
def should_use_global_query(question: str) -> bool:
    """
    Xác định xem câu hỏi có nên dùng global query không
    Dựa trên các từ khóa mang tính tổng quát/khái quát
    """
    keywords = [
    # Các cụm từ phổ biến
    "tóm tắt", "tổng quát", "khái quát", "tổng quan",
    "đại khái", "tóm gọn", "nhìn chung", "mô tả chung",
    "toàn cảnh", "bao quát", "khái lược", "phác thảo",
    "sơ lược", "nội dung chính", "nội dung tổng quát",
    "tổng hợp", "hệ thống lại", "cái nhìn tổng thể",
    "bức tranh chung", "góc nhìn chung", "cấu trúc chung",
    "nói chung", "chủ đề chính", "cấu trúc tổng quát",
    "gợi ý chung",
    "nội dung cốt lõi", "trình bày ngắn gọn", "nền tảng chung",
    "khái niệm tổng thể", "cốt lõi là gì", "giới thiệu tổng quát",
    "giới thiệu sơ lược", "giới thiệu chung", "đại cương",
    "khung chương trình", "khung nội dung", "mục tiêu chung",
    "mục tiêu tổng thể", "định hướng chung", "điểm nổi bật",
    "điểm chính", "ý chính", "trọng tâm", "xương sống", "nền tảng",
    "toàn thể", "tổng thể", "tầm nhìn chung", "mức độ khái quát",
    "ngắn gọn lại", "khái quát lại", "sơ đồ hóa", "liệt kê tổng quan",
    "bản đồ khái niệm", "trình bày đại cương", "trình bày sơ lược"
]
    normalized_question = question.lower()
    return any(kw in normalized_question for kw in keywords)


In [ ]:
should_use_global_query("Tóm tắt thông tin về chương trình đào tạo?")

In [ ]:
should_use_global_query("Điều kiện Đăng ký môn học gồm những gì?")

### Generate Answer

**Graph RAG**

In [ ]:
def generate_graph_rag_answers_from_csv(csv_path: str, output_path: str):
    df = pd.read_csv(csv_path)
    real_answers = []

    for i, row in df.iterrows():
        try:
            result = graph_answer(row["question"])
            real_answers.append(result)
        except Exception as e:
            real_answers.append(f"ERROR: {e}")

    df["real_answer"] = real_answers
    df.to_csv(output_path, index=False)
    return df


In [ ]:
generate_graph_rag_answers_from_csv("evaluation_data/Quy_dinh_dao_tao_Thac_si.csv", "evaluation_data/out_put_Quy_dinh_dao_tao_Thac_si.csv")

In [ ]:
def generate_llm_answers_from_csv(csv_path: str, output_path: str):

    df = pd.read_csv(csv_path)
    real_answers = []

    for i, row in df.iterrows():
        try:
            result = llm.invoke(row["question"])
            real_answers.append(result.content)
        except Exception as e:
            real_answers.append(f"ERROR: {e}")

    df["gpt_answer"] = real_answers
    df.to_csv(output_path, index=False)
    return df

In [ ]:
generate_llm_answers_from_csv("evaluation_data/Quy_dinh_dao_tao_Thac_si.csv", "evaluation_data/out_put_Quy_dinh_dao_tao_Thac_si_llm.csv")

### Evaluation

In [ ]:
def evaluate_qa_response(prediction, ground_truth, lang='vi'):
    """
    Đánh giá độ chính xác câu trả lời bằng BERTScore.
    
    Args:
        prediction (str): câu trả lời từ mô hình
        ground_truth (str): câu trả lời đúng
        lang (str): ngôn ngữ
        
    Returns:
        dict: {'exact_match': ..., 'bertscore': ...}
    """
    P, R, F1 = bert_score([prediction], [ground_truth], lang=lang, rescale_with_baseline=False)
    
    return {
        'bertscore_precision': round(P[0].item(), 4), # Trung bình độ tương đồng của mỗi token trong câu trả lời mô hình với token gần nhất trong đáp án chuẩn.
        'bertscore_recall': round(R[0].item(), 4), # Trung bình độ tương đồng của mỗi token trong đáp án chuẩn với token gần nhất trong câu trả lời mô hình.
        'bertscore_f1': round(F1[0].item(), 4) # Trung bình hài hòa giữa Precision và Recall.
    }


In [ ]:
def calculate_bert_score(file_path, output_path):
    # Đọc dữ liệu từ file
    df = pd.read_csv(file_path)
    
    # Tính BERTScore cho từng dòng
    results = df.apply(lambda row: evaluate_qa_response(row['answer'], row['real_answer']), axis=1)
    results_df = pd.DataFrame(results.tolist())
    
    # Ghép kết quả vào DataFrame gốc
    df = pd.concat([df, results_df], axis=1)
    
    # Tính điểm trung bình toàn bộ
    print("Trung bình BERTScore:")
    print("Precision:", df['bertscore_precision'].mean())
    print("Recall   :", df['bertscore_recall'].mean())
    print("F1       :", df['bertscore_f1'].mean())
    
    # Ghi ra file nếu muốn lưu lại
    df.to_csv(output_path, index=False)

In [ ]:
calculate_bert_score("evaluation_data/out_put_Quy_dinh_dao_tao_Thac_si_llm.csv", "evaluation_data/output_with_bertscore_llm.csv")

### UI Demo

In [ ]:
import tkinter as tk
from tkinter import Canvas, Frame, Scrollbar

# ==== GUI Functions ====

def send_message():
    user_input = entry.get()
    if user_input.strip():
        create_bubble(user_input, sender="user")
        entry.delete(0, tk.END)

        # Hiện loading và giữ lại label
        loading_label = create_bubble("Đang tạo câu trả lời...", sender="bot", return_label=True)

        def generate_and_replace():
            answer = global_answer(user_input) if should_use_global_query(user_input) else graph_answer(user_input)
            update_bubble(loading_label, answer)

        root.after(100, generate_and_replace)

def create_bubble(text, sender="bot", return_label=False):
    bubble_frame = Frame(message_frame, bg="#F0F0F0")

    msg_label = tk.Label(
        bubble_frame,
        text=text,
        wraplength=400,
        justify="left",
        font=("Arial", 12),
        padx=12,
        pady=6,
        bd=0
    )

    if sender == "user":
        msg_label.config(bg="#5C6BC0", fg="white")
        msg_label.pack(side="right", padx=5, pady=4)
        bubble_frame.pack(anchor="e", fill="x", padx=(50, 10), pady=4)
    else:
        msg_label.config(bg="#ffffff", fg="black")
        msg_label.pack(side="left", padx=5, pady=4)
        bubble_frame.pack(anchor="w", fill="x", padx=(10, 50), pady=4)

    canvas.update_idletasks()
    canvas.yview_moveto(1.0)

    return msg_label if return_label else None

def update_bubble(label, new_text):
    label.config(text=new_text)
    canvas.update_idletasks()
    canvas.yview_moveto(1.0)

# ==== Main window ====
root = tk.Tk()
root.title("Chatbot Demo UI")
root.geometry("600x650")
root.configure(bg="#F0F0F0")

# Root grid config
root.rowconfigure(0, weight=1)
root.rowconfigure(1, weight=0)
root.columnconfigure(0, weight=1)
root.columnconfigure(1, weight=0)

# ==== Display Area ====
display_frame = Frame(root, bg="#F0F0F0")
display_frame.grid(row=0, column=0, columnspan=2, sticky="nsew")

canvas = Canvas(display_frame, bg="#F0F0F0", highlightthickness=0)
canvas.pack(side="left", fill="both", expand=True)

scrollbar = Scrollbar(root, command=canvas.yview)
scrollbar.grid(row=0, column=2, sticky="ns")
canvas.configure(yscrollcommand=scrollbar.set)

scrollable_frame = Frame(canvas, bg="#F0F0F0")
message_frame = Frame(scrollable_frame, bg="#F0F0F0")

canvas_window = canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
message_frame.pack(fill="both", expand=True)

# Buộc width của scrollable_frame = width của canvas
def resize_canvas(event):
    canvas.itemconfig(canvas_window, width=event.width)

canvas.bind("<Configure>", resize_canvas)

scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# ==== Input Area ====
entry_frame = Frame(root, bg="#FFFFFF", bd=1)
entry_frame.grid(row=1, column=0, columnspan=2, sticky="ew")
entry_frame.columnconfigure(0, weight=1)

entry = tk.Entry(entry_frame, font=("Arial", 12), bd=0)
entry.grid(row=0, column=0, sticky="ew", padx=(10, 0), pady=10)
entry.bind("<Return>", lambda event: send_message())  # Enter to send

send_btn = tk.Button(entry_frame, text="➤", command=send_message,
                     font=("Arial", 14), bg="#FFFFFF", fg="#5C6BC0", bd=0)
send_btn.grid(row=0, column=1, sticky="e", padx=10, pady=10)

# ==== Start App ====
root.mainloop()
